## Waiting model

according to Waiting in crowded places: influence of number of pedestrians, waiting time and obstacles,
[10.1098/rsif.2023.0193](https://royalsocietypublishing.org/doi/10.1098/rsif.2023.0193)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import normal
import pathlib
import jupedsim as jps
import pedpy
from jupedsim.internal.notebook_utils import read_sqlite_file, animate
from shapely import Polygon, Point
from typing import List, Tuple
from shapely.affinity import translate
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import gaussian_kde

In [ ]:
cell_size = 0.5
num_agents = 40
direction = "right" # "left" or "right" where the agents are entering the area
#---------------
Bahnsteig_exp= Polygon([(0.5,3.5),(0.5,-3.5,),(-19.5,-3.5),(-19.5,3.5)])
Wand = Polygon([(-7.75,-0.2),(-11.35,-0.2),(-11.35,0.4),(-7.75,0.4)])
Haus = Polygon([(-7.75,-1.4),(-11.35,-1.4),(-11.35,1.6),(-7.75,1.6)])
Haus = Polygon([(-7.75,-1.4),(-11.35,-1.4),(-11.35,1.6),(-10.35, 1.6), (-10.35, 0.6), (-8.75,0.6), (-8.75, 1.6), (-7.75, 1.6)])

#area = Bahnsteig_exp
Wand = translate(Wand, xoff=19.5, yoff=4.07)
Haus = translate(Haus, xoff=19.5, yoff=4.07)
area = translate(Bahnsteig_exp, xoff=19.5, yoff=4.07)
center = Wand.centroid.coords[0]  # center is a tuple (x, y)
radius = 2
angles = np.linspace(0, 2 * np.pi, 7)[:-1]  # 6 points; the 7th equals the first
hexagon_coords = [
    (center[0] + radius * np.cos(theta), center[1] + radius * np.sin(theta))
    for theta in angles
]
hexagon = Polygon(hexagon_coords)
print(hexagon_coords)

obstacles = []
obstacle_type = "Wand"
assert obstacle_type in ["Wand", "Haus", "Hexagon"], "obstacle_type must be one of ['Wand', 'Haus', 'Hexagon']"
if obstacle_type == "Wand":
    print("Wand")
    obstacles = [Wand]
elif obstacle_type == "Haus":
    print("Haus")
    obstacles = [Haus]
elif obstacle_type == "Hexagon":
    print("Hexagon")
    obstacles = [hexagon]    


for obstacle in obstacles:
    print(">> ", obstacle)
    area = area.difference(obstacle)

spawning_area = Polygon(
    [
        (0.0, 5.49),
        (-20, 5.49),
        (-20, 2.6),
        (0.0, 2.6),        
    ]
)
spawning_area = translate(spawning_area, xoff=39.5, yoff=0)
waiting_area = Polygon([(0.0, 7.57), (20.0, 7.57), (20.0, 0.56), (0.0, 0.56)])
exit_areas = [
    Polygon([(5, 0), (5, 0.6), (6.4, 0.6), (6.4, 0)]),
    Polygon([(10, 0), (10, 0.6), (11.4, 0.6), (11.4, 0)]),
    Polygon([(15, 0), (15, 0.6), (16.3, 0.6), (16.3, 0.0)]),
]
area = area.union(exit_areas[0]).union(exit_areas[1]).union(exit_areas[2]).union(spawning_area)


spawning_area = Polygon(
    [
        (28, 5.49),
        (39, 5.49),
        (39, 2.6),
        (28.0, 2.6),        
    ]
)
pos_in_spawning_area = jps.distributions.distribute_by_number(
    polygon=spawning_area,
    number_of_agents=num_agents,
    distance_to_agents=0.4,
    distance_to_polygon=0.4,
    seed=13,
)

fig, ax = plt.subplots(figsize=(12, 12))
plt.plot(*area.exterior.xy, color="black")
plt.fill(*spawning_area.exterior.xy, color="lightsteelblue", label="Spawning area")
plt.fill(*waiting_area.exterior.xy, color="darkseagreen", label="Waiting area")
plt.scatter(*zip(*pos_in_spawning_area))

for i, interior in enumerate(area.interiors):
    if i == 0:
        plt.fill(*interior.xy, color="gray", alpha=0.9, label="Obstacle")
    else:
        plt.fill(*interior.xy, color="gray", alpha=0.9)

for i, exit in enumerate(exit_areas):
    if i == 0:
        plt.fill(*exit.exterior.xy, color="indianred", label="Destination")
    else:
        plt.fill(*exit.exterior.xy, color="indianred")
# plt.scatter(*zip(*koordinaten), color='lightcoral', label='waiting points')
ax.set_aspect("equal")

plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.1), ncol=4)
plt.savefig(f"waitigpointsmap_{num_agents}_{obstacle_type}.png", bbox_inches="tight")
plt.show()

In [ ]:
def is_valid_point(new_point, existing_points, min_distance, obstacles=None):
    """Überprüft, ob der neue Punkt den Mindestabstand zu allen vorhandenen Punkten einhält.

    und sich nicht innerhalb eines oder mehrerer obstacles befindet.
    """
    if obstacles is not None:
        
        # Wenn obstacles eine Liste oder ein Tupel von Polygonen ist, iteriere durch alle
        if isinstance(obstacles, (list, tuple)):
            for obs in obstacles:
                if obs.distance(Point(new_point)) <= min_distance:
                     return False

                if obs.contains(Point(new_point)):
                    return False
        else:
            # Wenn obstacles nur ein einzelnes Polygon ist
            if obstacles.distance(Point(new_point)) <= min_distance:
                return False

            if obstacles.contains(Point(new_point)):
                return False
            

    for point in existing_points:
        if np.linalg.norm(np.array(new_point) - np.array(point)) < min_distance:
            return False
    return True

In [ ]:
def distance_cost(x, L=20, direction="right"):
    """distance to entrance cost function notmalized to 1 (D).
    
    direction: where the agents are coming from.
    """
    assert direction in ["right", "left"], "direction must be either 'right' or 'left'."
    if direction == "right":        
        return (x / L) ** 3 - 1
    else:
        return 1 - (x / L) ** 3


def attractiveness(y, max_y, a=0.3):
    """
    Calculate the normalized attractiveness based on the waiting position y (T).

    Parameters:
      y (float or array-like): The position along the platform.
      a (float): A positive constant controlling the decay rate.
      max_y (float): The maximum y value on the platform (opposite side).

    Returns:
      float or array-like: Normalized attractiveness, with T(0)=1 and T(max_y)=0.
    """
    f_y = 1 / (1 + np.exp(-a * y))
    f_max = 1 / (1 + np.exp(-a * max_y))
    # f(0) is 1/(1+exp(0)) = 0.5
    return (f_y - f_max) / (0.5 - f_max)


def hazard_repulsion(
    X, Y, safety_line_x=19, safety_line_y1=1.56, safety_line_y2=6.57, b=4, direction="right"
):
    """
    Compute the repulsion effect from the platform edge based on the distance to the safety line.

    The effective distance to the safety line is taken as the minimum among:
      - The distance from the safety line in the x-direction (safety_line_x - X)
      - The distance from the lower safety line (Y - safety_line_y1)
      - The distance from the upper safety line (safety_line_y2 - Y)

    The repulsion is then calculated using:
        E = 1 / (1 + exp(-b * distance)) - 1

    Parameters:
      X : float or array_like
          X-coordinate(s) of the waiting position.
      Y : float or array_like
          Y-coordinate(s) of the waiting position.
      safety_line_x : float, optional
          X-coordinate of the safety line (default is 19).
      safety_line_y1 : float, optional
          Lower Y-bound of the safety line (default is 1.56).
      safety_line_y2 : float, optional
          Upper Y-bound of the safety line (default is 6.57).
      b : float, optional
          Decay constant controlling the exponential drop-off (default is 4).

    Returns:
      E : float or array_like
          The repulsion effect. Close to the safety line the effect is strong (more negative),
          and it decays exponentially to 0 further into the platform.
    """
    assert direction in ["right", "left"], "direction must be either 'right' or 'left'."
    if direction == "right":
        direction_factor = -1
    else:
        direction_factor = 1

    # Compute distances from the safety line boundaries
    dist_x = (safety_line_x - X) * direction_factor
    dist_y_lower = Y - safety_line_y1
    dist_y_upper = safety_line_y2 - Y

    # Effective distance is the minimum of the three distances
    dist_to_safety_line = np.minimum(dist_x, np.minimum(dist_y_lower, dist_y_upper))

    # Compute the repulsion effect using the sigmoid form and shift it so that far from the edge E=0
    E = 1 / (1 + np.exp(-b * dist_to_safety_line)) - 1

    return E



def flow_avoidance(X, Y, c=10, d=3, e=1.5, x0=0, y0=4):
    """
    Compute the normalized flow avoidance field over a grid of positions.

    Parameters:
        X, Y : float or array_like
            The x and y coordinates of the waiting positions.
        c : float, optional
            Magnitude constant for the flow avoidance effect (default is 10).
        d : float, optional
            Spread parameter in the x-direction (default is 3).
        e : float, optional
            Spread parameter in the y-direction (default is 1.5).
        x0 : float, optional
            x-coordinate of the entrance (default is 0).
        y0 : float, optional
            y-coordinate of the entrance (default is 4).

    Returns:
        F : float or array_like
            Normalized flow avoidance field. The maximum repulsion is -1 at the entrance,
            and it decays toward 0 further away.
    """
    # Calculate the raw flow avoidance field
    F_raw = -c * np.exp(-(((X - x0) ** 2) / d**2 + ((Y - y0) ** 2) / e**2))

    # Normalize so that the maximum repulsion (most negative value) becomes -1
    F_min = np.min(F_raw)
    F_normalized = -F_raw / F_min

    return F_normalized


def obstacle_floor_field(X, Y, x1, x2, y1, y2, f, g, h, boarding_side="lower"):
    """
    Compute the floor field for a stationary obstacle.

    The obstacle has a two-sided effect:
      - Areas in front of the obstacle (with y-values above the obstacle's vertical midpoint)
        become attractive.
      - Areas behind the obstacle (with y-values below the vertical midpoint) are repulsive.
    The influence of the obstacle decays with distance.

    Parameters:
        boarding_side : str, optional
        X : float or array_like
            The x-coordinate(s) of the waiting position(s).
        Y : float or array_like
            The y-coordinate(s) of the waiting position(s).
        x1, x2 : float
            The x-coordinates of the obstacle's corners.
        y1, y2 : float
            The y-coordinates of the obstacle's corners.
        f : float
            Constant scaling the magnitude of the obstacle effect.
        g : float
            Scaling parameter controlling decay in the x-direction.
        h : float
            Scaling parameter controlling decay in the y-direction.

    Returns:
        O : float or array_like
            The resulting floor field value. Positive values indicate attractive areas
            (in front of the obstacle) while negative values indicate repulsive areas
            (behind the obstacle). The effect decays to zero with distance.
    """
    if boarding_side == "lower":
        # Attractive side at y1: map y so that y1 -> 1 and y2 -> -1.
        A = 2 * (y2 - Y) / (y2 - y1) - 1
        #A = (Y - (y1 - y2)/2)
    elif boarding_side == "upper":
        # Attractive side at y2: map y so that y1 -> -1 and y2 -> 1.
        A = 2 * (Y - y1) / (y2 - y1) - 1
    else:
        raise ValueError("boarding_side must be either 'lower' or 'upper'.")

    # Exponential decay based on the distance from the obstacle.
    exponent = -(((X - x1) * (X - x2)) / (g**2) + ((Y - y1) * (Y - y2)) / (h**2))
    O_raw = f * A * np.exp(exponent)


    # Normalize to [-1, 1]
    max_abs = np.max(np.abs(O_raw))
    if max_abs != 0:
        O_normalized = O_raw / max_abs
    else:
        O_normalized = O_raw

    return O_normalized
    

def obstacle_floor_field2(X, Y, x1, x2, y1, y2, f, g, h, boarding_side="lower"):
    """
    Compute the floor field for a stationary obstacle.

    The obstacle has a two-sided effect:
      - Areas in front of the obstacle (with y-values above the obstacle's vertical midpoint)
        become attractive.
      - Areas behind the obstacle (with y-values below the vertical midpoint) are repulsive.
    The influence of the obstacle decays with distance.

    Parameters:
        boarding_side : str, optional
        X : float or array_like
            The x-coordinate(s) of the waiting position(s).
        Y : float or array_like
            The y-coordinate(s) of the waiting position(s).
        x1, x2 : float
            The x-coordinates of the obstacle's corners.
        y1, y2 : float
            The y-coordinates of the obstacle's corners.
        f : float
            Constant scaling the magnitude of the obstacle effect.
        g : float
            Scaling parameter controlling decay in the x-direction.
        h : float
            Scaling parameter controlling decay in the y-direction.

    Returns:
        O : float or array_like
            The resulting floor field value. Positive values indicate attractive areas
            (in front of the obstacle) while negative values indicate repulsive areas
            (behind the obstacle). The effect decays to zero with distance.
    """

    O_raw = -((f * (Y + (y1 - y2)/2)) * np.exp(
        (-(X - x1) * (X - x2) / (g ** 2 ))-
        (Y - y1) * (Y - y2) / (h ** 2)))


    # Normalize to [-1, 1]
    max_abs = np.max(np.abs(O_raw))
    if max_abs != 0:
        O_normalized = O_raw / max_abs
    else:
        O_normalized = O_raw

    return O_normalized
    

def obstacle_field_at_point(point, obstacle, f, L, boarding_side='lower'):
    """
    Compute the obstacle field at a single point based on the distance to the obstacle
    and a smooth sign factor (attractive vs. repulsive side).

    Parameters:
        point (shapely.geometry.Point): The point where the field is computed.
        obstacle (shapely.geometry.Polygon): The obstacle.
        f (float): Scaling constant for the obstacle effect.
        L (float): Decay length controlling how quickly the effect decreases with distance.
        boarding_side (str): 'lower' or 'upper', indicating which edge is attractive.

    Returns:
        float: The obstacle field value at the given point.
    """
    # Distance-based approach, for example:
    #  O = f * s(y) * exp(-dist/L)
    # where dist is the distance to the obstacle, and s(y) sets sign based on 'boarding_side'.
    

    assert L > 0, "L must be greater than 0."
    dist = point.distance(obstacle)

    # Get obstacle bounds
    x_min, y_min, x_max, y_max = obstacle.bounds

    # Smooth sign factor:
    if boarding_side == 'lower':
        # If the point is near y_min, we want +1 (attractive).
        # If the point is near y_max, we want -1 (repulsive).
        # Here we use a simple sigmoid that transitions around y_min.
        s = 2.0 / (1.0 + np.exp(0.5 * (point.y - y_min))) - 1.0
    elif boarding_side == 'upper':
        # Reverse the logic for the upper edge
        s = 2.0 / (1.0 + np.exp(0.5 * (y_max - point.y))) - 1.0
    else:
        raise ValueError("boarding_side must be either 'lower' or 'upper'.")

 
    decay = np.exp(-dist / L)

    return f * s * decay

def compute_obstacle_field_grid_multi(X, Y, obstacles, f, L, boarding_side='lower'):
    """
    Compute the total obstacle field over a grid (X, Y) by summing contributions 
    from multiple obstacles, then normalize the result to [-1, +1].

    Parameters:
        X, Y (np.array): 2D arrays (from np.meshgrid) of the grid coordinates.
        obstacles (list): A list of Shapely Polygons or coordinate lists representing obstacles.
        f (float): Scaling constant for the obstacle effect (applied uniformly to all obstacles).
        L (float): Decay length controlling how quickly the effect decreases with distance.
        boarding_side (str): 'lower' or 'upper', indicating which edge is attractive.

    Returns:
        np.array: The computed obstacle field on the grid, normalized to the range [-1, 1].
    """
    # Prepare an array for the total field
    field = np.zeros_like(X, dtype=float)

    polygons = []
    for obs in obstacles:
        if isinstance(obs, Polygon):
            polygons.append(obs)
        else:
            polygons.append(Polygon(obs))

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            pt = Point(X[i, j], Y[i, j])
            total_value = 0.0
            for poly in polygons:
                total_value += obstacle_field_at_point(pt, poly, f, L, boarding_side)

            field[i, j] = total_value

    max_abs = np.max(np.abs(field))
    if max_abs > 0:
        field /= max_abs

    return field




def compute_combined_obstacle_field(
    X, Y, obstacles, f_param=2, g_param=3, h_param=1.5, boarding_side="lower"
):
    """
    Compute the combined obstacle floor field over a grid from one or more obstacles.

    Parameters:
        X, Y : np.array
            Grid coordinates (e.g., created by np.meshgrid).
        obstacles : list
            A list of obstacles. Each obstacle can either be a Shapely Polygon or
            a list of coordinate tuples defining the obstacle.
        f_param : float
            Scaling constant for the obstacle effect.
        g_param : float
            Decay parameter in the x-direction.
        h_param : float
            Decay parameter in the y-direction.
        boarding_side : str
            'lower' or 'upper', indicating which side is attractive.

    Returns:
        np.array: The combined obstacle floor field on the grid.
    """
    O_total = np.zeros_like(X, dtype=float)
    for obs in obstacles:
        if isinstance(obs, Polygon):
            obstacle_coords = list(obs.exterior.coords)
        else:
            obstacle_coords = obs

        x_coords, y_coords = zip(*obstacle_coords)
        x1, x2 = min(x_coords), max(x_coords)
        y1, y2 = min(y_coords), max(y_coords)
        print(f"{x1=}, {x2=}, {y1=}, {y2=}")

        O_total += obstacle_floor_field(
            X, Y, x1, x2, y1, y2, f_param, g_param, h_param, boarding_side
        )

    return O_total


def compute_probability_map(D, T, E, F, O, w1=1, w2=2, w3=3, w4=1, w5=3):
    """
    Combine various floor fields into an overall attractiveness map and normalize
    it to produce a probability distribution.

    Parameters:
        D (np.array): Floor field for distance cost.
        T (np.array): Floor field for waiting attractiveness.
        E (np.array): Floor field for edge repulsion.
        F (np.array): Floor field for flow avoidance.
        O (np.array): Floor field for obstacle effect.
        w1, w2, w3, w4, w5 (float): Weights for D, T, E, F, and O, respectively.
            Default values are w1=1, w2=2, w3=3, w4=1, w5=3.

    Returns:
        np.array: A probability distribution over the area, normalized such that
                  the values sum to 1.
    """
    S = w1 * D + w2 * T + w3 * E + w4 * F + w5 * O

    # Shift the map so that the minimum value is 0.
    S_shifted = S - np.min(S)

    # Normalize the map so that the sum equals 1.
    probabilities = S_shifted / np.sum(S_shifted)

    return probabilities

In [ ]:
def oriented_obstacle_field_at_point(
    point: Point,
    obstacle: Polygon,
    front_direction: np.ndarray,
    attract_back: float,
    repel_front: float,
    decay_length: float
) -> float:
    """
    Compute the obstacle effect at a single point, assuming the obstacle has a known orientation.

    Parameters:
      point (Point)           : The query point.
      obstacle (Polygon)      : The obstacle geometry (any shape, though typically rectangular).
      front_direction (array) : A 2D unit vector indicating which way is 'front' of the obstacle.
      attract_back (float)    : Positive magnitude for the behind side (attractive).
      repel_front (float)     : Positive magnitude for the front side (repulsive).
      decay_length (float)    : Distance scale controlling how fast the effect decays.

    Returns:
      float: The obstacle field value at 'point'.
             Positive values => attractive, negative => repulsive.
    """
    assert decay_length > 0, "decay_length must be positive."

    # 1) Distance-based decay
    dist = point.distance(obstacle)
    decay_factor = np.exp(-dist / decay_length)

    # 2) Determine whether 'point' is in front or behind the obstacle
    #    We'll compare the vector from the obstacle center to 'point' against front_direction.
    center = obstacle.centroid
    vec_to_point = np.array([point.x - center.x, point.y - center.y])
    dot = np.dot(vec_to_point, front_direction)

    if dot < 0:
        # front side => positiv effect
        raw_value = -repel_front
    else:
        raw_value = attract_back

    return raw_value * decay_factor

def compute_oriented_obstacle_field_grid(
    X: np.ndarray,
    Y: np.ndarray,
    obstacles: list,
    front_directions: list,
    attract_back=1.0,
    repel_front=1.0,
    decay_length=2.0
) -> np.ndarray:
    """
    Compute the sum of oriented obstacle fields over a grid for multiple obstacles.

    Parameters:
      X, Y (np.array)       : 2D meshgrid coordinates.
      obstacles (list)      : A list of shapely Polygons (or coordinate lists).
      front_directions (list):
                              A list of 2D unit vectors (np.array([dx, dy])) 
                              indicating the 'front' direction for each obstacle.
      attract_back (float)  : Magnitude of attraction behind obstacle.
      repel_front (float)   : Magnitude of repulsion in front of obstacle.
      decay_length (float)  : Distance scale for exponential decay.

    Returns:
      np.array: The resulting obstacle field (same shape as X, Y).
    """
    field = np.zeros_like(X, dtype=float)

    # Ensure each obstacle is a Polygon
    poly_list = []
    for obs in obstacles:
        if isinstance(obs, Polygon):
            poly_list.append(obs)
        else:
            poly_list.append(Polygon(obs))

    # Loop over each cell in the grid
    nrows, ncols = X.shape
    for i in range(nrows):
        for j in range(ncols):
            pt = Point(X[i, j], Y[i, j])
            val = 0.0
            # Sum contributions from all obstacles
            for k, poly in enumerate(poly_list):
                # The k-th front_direction corresponds to the k-th obstacle
                val += oriented_obstacle_field_at_point(
                    pt, 
                    poly, 
                    front_directions[k], 
                    attract_back, 
                    repel_front, 
                    decay_length
                )
            field[i, j] = val

    max_abs = np.max(np.abs(field))
    if max_abs > 1e-12:
        field /= max_abs

    return field

In [ ]:
#====================
f_param = 1.25  # scaling constant for the obstacle effect
g_param = 5  # decay parameter in x-direction
h_param = 2  # decay parameter in y-direction

w1=1
w2=0
w3=0
w4=0
w5=0
#====================
min_x, min_y, max_x, max_y = waiting_area.bounds
x_cells = np.arange(
    min_x + cell_size / 2, max_x, cell_size
)  # Liste mit Zell-Mittelpunkten
y_cells = np.arange(
    min_y + cell_size / 2, max_y, cell_size
)  # Liste mit Zell-Mittelpunkten

X, Y = np.meshgrid(x_cells, y_cells)

safety_line_x = max_x - 1 if direction == "left" else min_x + 1
x0_entrance = 0 if direction == "left" else 20
y0_entrance = 4
X_min = max_x - min_x
D = distance_cost(X, L=X_min, direction=direction)
T = attractiveness(Y, max_y)
E = hazard_repulsion(
    X, Y, safety_line_x=safety_line_x, safety_line_y1=min_y + 1, safety_line_y2=max_y - 1, b=4, direction=direction
)
F = flow_avoidance(X, Y, c=10, d=3, e=1.5, x0=x0_entrance, y0=y0_entrance)


O = compute_combined_obstacle_field(
    X, Y, obstacles, f_param, g_param, h_param, boarding_side="lower"
)
L=2
O2 = compute_obstacle_field_grid_multi(X, Y, obstacles, f_param, L, boarding_side='lower')
front_vec = np.array([0.0, -1.0])
front_dirs = [front_vec for _ in obstacles]
O3 = compute_oriented_obstacle_field_grid(
        X, Y,
        obstacles,
        front_dirs,
        attract_back=1,
        repel_front=1,
        decay_length=1.5
    )

probabilities = compute_probability_map(D, T, E, F, O, w1=w1, w2=w2, w3=w3, w4=w4, w5=w5)
probabilities2 = compute_probability_map(D, T, E, F, O3, w1=w1, w2=w2, w3=w3, w4=w4, w5=w5)
names = ["D", "T", "E", "F", "O", "probabilities"]
names = ["T", "E", "F","O", "O3", "probabilities", "probabilities2"]
values = [D, T, E, F, O, probabilities]
values = [T, E, F, O,O3,probabilities, probabilities2]
for i, (name, value) in enumerate(zip(names, values)):
    fig, ax = plt.subplots(figsize=(12, 6))
    contour = ax.contourf(X, Y, value, levels=50, cmap="jet")  # vmin=-2, vmax=2
    cbar = fig.colorbar(
        contour, ax=ax, fraction=0.046, pad=0.04, shrink=0.5, label=name
    )
    if name == "probabilities":
        cbar.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{x*1e3:.1f}×10³"))        
        cbar.ax.yaxis.get_offset_text().set_fontsize(18)  # Adjust font size

    x, y = waiting_area.exterior.xy
    ax.plot(x, y, "k-", lw=1)

    for polygon in obstacles:
        ax.fill(*polygon.exterior.xy, color="black", edgecolor="white", alpha=0.9)

    ax.set_aspect("equal")  # No need to set this twice
    plt.savefig(f"Fields_{name}_{obstacle_type}_w1_{w1}_w2_{w2}_w3_{w3}_w4_{w4}_w5_{w5}.png", bbox_inches="tight")
    # plt.close(fig)  # Close the figure to free memory


if True:
    threshold = 0.2 * np.max(probabilities)  # Set a cutoff at 10% of the max value
    probabilities[probabilities < threshold] = 0  # Remove very low probabilities
    probabilities /= np.sum(probabilities)  # Renormalize

probabilities_flat = probabilities.ravel()


indices = np.arange(probabilities_flat.size)

num_points = num_agents
max_attempts = 100  # Maximale Anzahl der Versuche
min_distance = 0.25  # Mindestabstand


selected_points = []

for _ in range(num_points):
    attempts = 0
    while attempts < max_attempts:
        selected_index = np.random.choice(indices, p=probabilities_flat)
        selected_index_2d = np.unravel_index(selected_index, probabilities.shape)
        candidate_point = (X[selected_index_2d], Y[selected_index_2d])

        if is_valid_point(
            candidate_point, selected_points, min_distance, obstacles=obstacles
        ):
            selected_points.append(candidate_point)
            break
        attempts += 1

    if attempts == max_attempts:
        print("Skipping point due to max_attempts exhaustion")

selected_points = np.array(selected_points)

fig, ax = plt.subplots(figsize=(12, 6))
contour = ax.contourf(X, Y, probabilities, levels=10, cmap="jet")
cbar = fig.colorbar(contour, ax=ax, label="Probability", shrink=0.5)
cbar.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{x*1e3:.1f}×10³"))        
cbar.ax.yaxis.get_offset_text().set_fontsize(18)  # Adjust font size

x, y = waiting_area.exterior.xy
ax.plot(x, y, "k-", lw=1)
for polygon in obstacles:
    ax.fill(*polygon.exterior.xy, color="black", edgecolor="white", linewidth=2.5, alpha=0.7)


ax.scatter(
    selected_points[:, 0],
    selected_points[:, 1],
    c="red",
    s=10,
    edgecolor="black",
    label="Ausgewählte Punkte",
)
ax.set_aspect("equal")
# -----------------------------
# Create new axes on top and right for marginal distributions
divider = make_axes_locatable(ax)
ax_top = divider.append_axes("top", 1.2, pad=0.1, sharex=ax)
ax_right = divider.append_axes("right", 1.2, pad=0.1, sharey=ax)

# Remove tick labels where axes overlap
plt.setp(ax_top.get_xticklabels(), visible=False)
plt.setp(ax_right.get_yticklabels(), visible=False)

# ---------------
# X-distribution (top)
x_data = selected_points[:, 0]
x_density = gaussian_kde(x_data)
x_vals = np.linspace(x_data.min(), x_data.max(), 200)
ax_top.plot(x_vals, x_density(x_vals), color="blue")
ax_top.fill_between(x_vals, x_density(x_vals), color="blue", alpha=0.3)
ax_top.set_ylabel("Density")

# ---------------
# Y-distribution (right)
y_data = selected_points[:, 1]
y_density = gaussian_kde(y_data)
y_vals = np.linspace(y_data.min(), y_data.max(), 200)
ax_right.plot(y_density(y_vals), y_vals, color="green")
ax_right.fill_betweenx(y_vals, 0, y_density(y_vals), color="green", alpha=0.3)
ax_right.set_xlabel("Density")

plt.tight_layout()

plt.savefig(f"Verteilung_{num_agents}_{obstacle_type}_w1_{w1}_w2_{w2}_w3_{w3}_w4_{w4}_w5_{w5}.png",bbox_inches="tight")
#--------------

if True:
    threshold = 0.2 * np.max(probabilities2)  # Set a cutoff at 10% of the max value
    probabilities2[probabilities2 < threshold] = 0  # Remove very low probabilities
    probabilities2 /= np.sum(probabilities2)  # Renormalize

probabilities_flat2 = probabilities2.ravel()


indices = np.arange(probabilities_flat2.size)

num_points = num_agents
max_attempts = 100 
min_distance = 0.25  
selected_points = []

for _ in range(num_points):
    attempts = 0
    while attempts < max_attempts:
        # Auswahl eines zufälligen Punktes basierend auf den Wahrscheinlichkeiten
        selected_index = np.random.choice(indices, p=probabilities_flat2)
        selected_index_2d = np.unravel_index(selected_index, probabilities2.shape)
        candidate_point = (X[selected_index_2d], Y[selected_index_2d])

        # Überprüfen des Abstands
        if is_valid_point(
            candidate_point, selected_points, min_distance, obstacles=obstacles
        ):
            selected_points.append(candidate_point)
            break
        attempts += 1

    if attempts == max_attempts:
        print("Skipping point due to max_attempts exhaustion")


selected_points = np.array(selected_points)

fig, ax = plt.subplots(figsize=(12, 6))
contour = ax.contourf(X, Y, probabilities2, levels=10, cmap="jet")
cbar = fig.colorbar(contour, ax=ax, label="Probability", shrink=0.5)
cbar.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{x*1e3:.1f}×10³"))        
cbar.ax.yaxis.get_offset_text().set_fontsize(18)  # Adjust font size

x, y = waiting_area.exterior.xy
ax.plot(x, y, "k-", lw=1)
for polygon in obstacles:
    ax.fill(*polygon.exterior.xy, color="black", edgecolor="white", linewidth=2.5, alpha=0.7)


ax.scatter(
    selected_points[:, 0],
    selected_points[:, 1],
    c="red",
    s=10,
    edgecolor="black",
    label="Ausgewählte Punkte",
)
ax.set_aspect("equal")
# -----------------------------
# Create new axes on top and right for marginal distributions
divider = make_axes_locatable(ax)
ax_top = divider.append_axes("top", 1.2, pad=0.1, sharex=ax)
ax_right = divider.append_axes("right", 1.2, pad=0.1, sharey=ax)

# Remove tick labels where axes overlap
plt.setp(ax_top.get_xticklabels(), visible=False)
plt.setp(ax_right.get_yticklabels(), visible=False)

# ---------------
# X-distribution (top)
x_data = selected_points[:, 0]
x_density = gaussian_kde(x_data)
x_vals = np.linspace(x_data.min(), x_data.max(), 200)
ax_top.plot(x_vals, x_density(x_vals), color="blue")
ax_top.fill_between(x_vals, x_density(x_vals), color="blue", alpha=0.3)
ax_top.set_ylabel("Density")

# ---------------
# Y-distribution (right)
y_data = selected_points[:, 1]
y_density = gaussian_kde(y_data)
y_vals = np.linspace(y_data.min(), y_data.max(), 200)
ax_right.plot(y_density(y_vals), y_vals, color="green")
ax_right.fill_betweenx(y_vals, 0, y_density(y_vals), color="green", alpha=0.3)
ax_right.set_xlabel("Density")

plt.tight_layout()

plt.savefig(f"Verteilung2_{num_agents}_{obstacle_type}_w1_{w1}_w2_{w2}_w3_{w3}_w4_{w4}_w5_{w5}.png",bbox_inches="tight")

In [ ]:
assert np.isclose(np.sum(probabilities), 1), "Summe der Wahrscheinlichkeiten ist nicht 1!"

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
contour = ax.contourf(X, Y, probabilities, levels=10, cmap="jet")
cbar = fig.colorbar(contour, ax=ax, label="Probability", shrink=0.5)
cbar.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{x*1e3:.1f}×10³"))        
cbar.ax.yaxis.get_offset_text().set_fontsize(18)  # Adjust font size

x, y = waiting_area.exterior.xy
ax.plot(x, y, "k-", lw=1)
for polygon in obstacles:
    ax.fill(*polygon.exterior.xy, color="black", edgecolor="white", linewidth=2.5, alpha=0.7)


# Zeichnen der ausgewählten Punkte auf der Karte
ax.scatter(
    selected_points[:, 0],
    selected_points[:, 1],
    c="red",
    s=10,
    edgecolor="black",
    label="Ausgewählte Punkte",
)
ax.set_aspect("equal")
# -----------------------------
# Create new axes on top and right for marginal distributions
divider = make_axes_locatable(ax)
ax_top = divider.append_axes("top", 1.2, pad=0.1, sharex=ax)
ax_right = divider.append_axes("right", 1.2, pad=0.1, sharey=ax)

# Remove tick labels where axes overlap
plt.setp(ax_top.get_xticklabels(), visible=False)
plt.setp(ax_right.get_yticklabels(), visible=False)

# ---------------
# X-distribution (top)
x_data = selected_points[:, 0]
x_density = gaussian_kde(x_data)
x_vals = np.linspace(x_data.min(), x_data.max(), 200)
ax_top.plot(x_vals, x_density(x_vals), color="blue")
ax_top.fill_between(x_vals, x_density(x_vals), color="blue", alpha=0.3)
ax_top.set_ylabel("Density")

# ---------------
# Y-distribution (right)
y_data = selected_points[:, 1]
y_density = gaussian_kde(y_data)
y_vals = np.linspace(y_data.min(), y_data.max(), 200)
ax_right.plot(y_density(y_vals), y_vals, color="green")
ax_right.fill_betweenx(y_vals, 0, y_density(y_vals), color="green", alpha=0.3)
ax_right.set_xlabel("Density")

# Layout anpassen und anzeigen
plt.tight_layout()

plt.savefig(f"Verteilung1_{num_agents}_{obstacle_type}_w1_{w1}_w2_{w2}_w3_{w3}_w4_{w4}_w5_{w5}.png")
plt.show()

# Waiting with Direct Steering

In [ ]:
def get_nearest_exit_and_journey_id(
    position: Point,
    exit_areas: List[Polygon],
    exit_ids: List[int],
    journey_ids: List[int],
) -> Tuple[int, int]:
    """
    Returns the nearest exit id and journey id to the given position.

    Parameters:
        position (Point): The current position.
        exit_areas (List[Polygon]): A list of exit areas.
        exit_ids (List[int]): A list of exit ids corresponding to the exit areas.
        journey_ids (List[int]): A list of journey ids corresponding to the exit areas.

    Returns:
        Tuple[int, int]: A tuple containing the nearest exit id and journey id.
    """
    min_distance = float("inf")
    best_index = None

    for i, exit_area in enumerate(exit_areas):
        distance = Point(position).distance(exit_area)
        if distance < min_distance:
            min_distance = distance
            best_index = i

    return exit_ids[best_index], journey_ids[best_index]

In [ ]:
trajectory_file = f"direct-steering_{num_agents}_{obstacle_type}.sqlite"  # output file
simulation = jps.Simulation(
    model=jps.CollisionFreeSpeedModel(
            strength_neighbor_repulsion=3,
            range_neighbor_repulsion=0.3,),
    geometry=area,
    trajectory_writer=jps.SqliteTrajectoryWriter(
        output_file=pathlib.Path(trajectory_file)
    ),
)

direct_steering_stage = simulation.add_direct_steering_stage()
direct_steering_journey = jps.JourneyDescription([direct_steering_stage])
direct_steering_journey_id = simulation.add_journey(direct_steering_journey)


exit_ids = []
journey_ids = []
for exit_area in exit_areas:
    exit_id = simulation.add_exit_stage(exit_area)
    exit_ids.append(exit_id)
    journey_id = simulation.add_journey(jps.JourneyDescription(exit_ids))
    journey_ids.append(journey_id)


v_distribution = normal(1.34, 0.05, num_agents)
wait_position = list(selected_points)
for counter, (pos, v0) in enumerate(zip(pos_in_spawning_area, v_distribution)):
    agent_id = simulation.add_agent(
        jps.CollisionFreeSpeedModelAgentParameters(
            journey_id=direct_steering_journey_id,
            stage_id=direct_steering_stage,
            position=pos,
            v0=v0,
        )
    )
    agent = simulation.agent(agent_id)
    agent.target = wait_position[counter]
    agent_position = Point(agent.position)
    agent_target = Point(agent.target)

    print(
        f"{counter}/{num_agents} time: {simulation.elapsed_time():.2f} s, Agent {agent_id} spawned at ({agent_position.x:.2f}, {agent_position.y:.2f}) with target ({agent_target.x:.2f}, {agent_target.y:.2f})"
    )
    max_spawning_count = 1000
    count = 0
    while agent_position.distance(agent_target) > 2 and count < max_spawning_count:
        count += 1
        simulation.iterate()    
        agent_position = Point(agent.position)


simulation.iterate(2000)

for agent in simulation.agents():
    exit_id, journey_id = get_nearest_exit_and_journey_id(
        agent.position, exit_areas, exit_ids, journey_ids
    )
    simulation.switch_agent_journey(agent.id, journey_id, exit_id)

while simulation.agent_count() > 0:
    simulation.iterate()

print(f"Simulation time: {simulation.elapsed_time()} s")

In [ ]:
trajectory_data, walkable_area = read_sqlite_file(trajectory_file)
animate(trajectory_data, walkable_area, every_nth_frame=20)

In [ ]:
cfsm_data, walkable_area = read_sqlite_file(trajectory_file)

fig = plt.figure(figsize=(20, 20))

ax2 = fig.add_subplot(122, aspect="equal")
pedpy.plot_trajectories(
    traj=cfsm_data,
    axes=ax2,
    walkable_area=pedpy.WalkableArea(area),
    color="red",
    traj_width=0.1,
)
#  waiting_positions is a list of selected_points where agents are waiting
wait_position = np.array(wait_position)
ax2.plot(wait_position[:, 0], wait_position[:, 1], "o", color="indianred", markersize=2)
fig.savefig(f"trajectory_{num_agents}_{obstacle_type}.png", bbox_inches="tight")
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax2 = fig.add_subplot(122, aspect="equal")

# Convert waiting positions to numpy array
wait_position = np.array(wait_position)

# Define colors
before_waiting_color = "indianred"
after_waiting_color = "gray"

# Iterate through each pedestrian in the trajectory data
for ped_id, traj in cfsm_data.data.groupby("id"):
    traj = traj.sort_values(by="frame")  # Ensure order by time
    
    # Find the first frame where the pedestrian is in a waiting position
    for _, wait_pos in enumerate(wait_position):
        distances = np.linalg.norm(traj[["x", "y"]].values - wait_pos, axis=1)
        wait_index = np.where(distances < threshold)[0]

        if len(wait_index) > 0:
            wait_frame = traj.iloc[wait_index[0]].frame
            break  # Stop searching once we find the waiting frame

    # Split the trajectory into before and after waiting
    traj_before_waiting = traj[traj.frame < wait_frame]
    traj_after_waiting = traj[traj.frame >= wait_frame]

    # Plot before waiting in blue
    ax2.plot(traj_before_waiting["x"], traj_before_waiting["y"], "-",color=before_waiting_color, linewidth=0.3)

    # Plot after waiting in red
    ax2.plot(traj_after_waiting["x"], traj_after_waiting["y"], color=after_waiting_color, linewidth=0.2, alpha=0.7)

# Plot waiting positions
ax2.plot(wait_position[:, 0], wait_position[:, 1], "o", color="indianred", markersize=2)
pedpy.plot_walkable_area(walkable_area=walkable_area, axes=ax2)
# Save and show
fig.savefig(f"trajectory2_{num_agents}_{obstacle_type}.png", bbox_inches="tight")
plt.show()
